## Import libraries

In [ ]:
import os
import jsonlines

import numpy as np
from collections import defaultdict

In [ ]:
result_dir = "../../../results"

## Correlation with consistency (bin chart)

In [ ]:
dataset_names = ["CommonsenseQA", "QASC", "100TFQA", "GSM8K"]
model_names = ["Llama-3.1-8B-Instruct", "gpt-4o-2024-11-20"]
prompting_strategies = ["zero-shot", "zero-shot-cot", "few-shot", "few-shot-cot"]

# Read score file
scores = {}
for model_name in model_names:
    for dataset_name in dataset_names:
        for prompting_strategy in prompting_strategies:
            # if dataset_name == "CommonsenseQA" and "gpt" in model_name and prompting_strategy == "zero-shot":
            #     pass
            # else:
            #     continue
            dd = defaultdict(list)

            output_dir = f"{result_dir}/{dataset_name}/{model_name}"
            predictions_path = os.path.join(output_dir, f"{prompting_strategy}_predictions.jsonl")
            raw_predictions_path = os.path.join(output_dir, f"{prompting_strategy}_raw_predictions.jsonl")
            try:
                with jsonlines.open(predictions_path) as fin:
                    id_predictions_map, id_consistency_map = {}, {}
                    for example in fin.iter():
                        id_predictions_map[example["id"]] = example["predictions"]
                        id_consistency_map[example["id"]] = example["consistency"]["mean"]
                X, Y, Z = [], [], []
                with jsonlines.open(raw_predictions_path) as fin:
                    for example in fin.iter():
                        confidences = []
                        for format_id, top_tokens in example["top_tokens"].items():
                            confidence = -1
                            for ii, top_tokenss in enumerate(top_tokens[::-1]):
                                if top_tokenss[0] == id_predictions_map[example["id"]][format_id]:
                                    if "top_probs" in example:
                                        confidence = example["top_probs"][format_id][-(ii+1)][0]
                                    else:
                                        confidence = np.exp(example["top_logprobs"][format_id][-(ii+1)][0])
                                    confidences.append(confidence)
                                    break
                        if len(confidences) != 8:
                            # print(example["top_tokens"])
                            # print(len(confidences))
                            pass
                            # raise Exception
                        else:
                            mean_confidence = np.mean(confidences)
                            X.append(mean_confidence)
                            Y.append(id_consistency_map[example["id"]])
                            Z.append(1.0*(id_consistency_map[example["id"]] >= 0.99))

                            # print(confidences)
                            # print()
                            id_predictions_map[example["id"]].pop('majority_voting')
                            # print(id_predictions_map[example["id"]])

                            for ii in range(8):
                                zz = list(id_predictions_map[example["id"]].values()).count(id_predictions_map[example["id"]][str(ii)])
                                dd[zz].append(confidences[ii])
                print(f"{dataset_name} / {model_name} / {prompting_strategy}")
                for zz in range(1, 9):
                    print(f"{zz} / {len(dd[zz]):4} / {np.mean(dd[zz]):.3f}")
                print()
            except:
                continue